<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### 활성화 함수
- \_\_init_\_() 함수에서는 모델에서 사용될 모듈(nn.Linear 등) 과 activation function (활성화 함수) 등을 정의함
- forward() 함수에서 실행되어야 하는 연산에 활성화 함수도 적용하면 됨
- 주요 활성화 함수
   - 시그모이드 함수 : nn.Sigmoid()
   - ReLU 함수 : nn.ReLU()
   - Leaky ReLU 함수 : nn.LeakyReLU()

In [17]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        # 명시적으로 인자명을 써주는 경우도 많이 쓰임
        self.linear = nn.Linear(in_features=input_dim, out_features=output_dim)
        self.activation = nn.Sigmoid() # 시그모이드 함수
        # self.activation = nn.LeakyReLU(0.1)
    def forward(self, x):
        return self.activation(self.linear(x))

In [18]:
x = torch.ones(4)  # input tensor
y = torch.zeros(3)  # expected output
model = LinearRegressionModel(4, 3)

# F.mse_loss() 대신에 nn.MSELoss() 도 동일한 기능을 하며, 주요 클래스가 nn namespace 에 있으므로, nn.MSELoss() 를 사용키로 함
loss_function = nn.MSELoss() 

In [19]:
learning_rate = 0.01
nb_epochs = 1000 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):

    y_pred = model(x)
    loss = loss_function(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [20]:
print(loss)
for param in model.parameters():
    print (param)

tensor(3.6109e-06, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.4156,  0.1900,  0.0570,  0.1866],
        [ 0.1644,  0.1956, -0.5121, -0.2394],
        [ 0.1067,  0.2290, -0.0424, -0.4314]], requires_grad=True)
Parameter containing:
tensor([-0.0180,  0.3916,  0.1052], requires_grad=True)


### 다층 레이어 구현
> raw level 로 구현해본 후, 좀더 유용한 클래스를 알아보기로 함
- input layer -> hidden layer -> output layer 순으로 순차적으로 작성해주면 됨
  - 내부 행렬곱 조건만 유의해주면 됨
- activation function 적용은 output layer 에는 적용하지 않는 것이 일반적임

In [21]:
import torch
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 10)
        self.linear2 = nn.Linear(10, 10)
        self.linear3 = nn.Linear(10, 10)
        self.linear4 = nn.Linear(10, output_dim)        
        self.activation = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        # |x| = (input_dim, output_dim)
        hidden = self.activation(self.linear1(x)) # |hidden| = (input_dim, 5)
        hidden = self.activation(self.linear2(hidden)) # |hidden| = (5, 5)        
        hidden = self.activation(self.linear3(hidden)) # |hidden| = (5, 5)                
        y = self.linear4(hidden) # 마지막 출력에는 activation 함수를 사용하지 않는 것이 일반적임
        return y

In [22]:
x = torch.ones(4)  # input tensor
y = torch.zeros(3)  # expected output
model = LinearRegressionModel(4, 3)
loss_function = nn.MSELoss()

In [23]:
learning_rate = 0.01
nb_epochs = 1000 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):

    y_pred = model(x)
    loss = loss_function(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [24]:
print(loss)
for param in model.parameters():
    print (param)

tensor(4.4134e-10, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.2381, -0.1646, -0.4914,  0.1632],
        [-0.2515,  0.0371, -0.2751,  0.3417],
        [ 0.3883,  0.2373,  0.1406,  0.2542],
        [ 0.0055,  0.3668, -0.1642,  0.2487],
        [-0.3894,  0.2901,  0.4721,  0.1022],
        [-0.0896,  0.3571,  0.4116,  0.1600],
        [-0.3992, -0.1270, -0.2315, -0.3204],
        [-0.4018,  0.0802, -0.3038,  0.1425],
        [-0.2204, -0.3152, -0.0634,  0.2391],
        [-0.3342,  0.4009, -0.1703, -0.2847]], requires_grad=True)
Parameter containing:
tensor([-0.3448,  0.4189,  0.2268, -0.2005, -0.4228, -0.2593, -0.4047, -0.0025,
         0.4826,  0.2634], requires_grad=True)
Parameter containing:
tensor([[-0.2806, -0.1956, -0.2087,  0.0386,  0.2737,  0.0701,  0.1040, -0.2926,
         -0.1926, -0.2716],
        [ 0.1223, -0.0471,  0.2707,  0.0010,  0.2944, -0.1528, -0.2384,  0.0343,
          0.1140, -0.2458],
        [ 0.1451, -0.1123,  0.0841,  0.0708,  0.0214,  0.0293

### nn.Sequential
- nn.Sequential 은 순서를 갖는 모듈의 컨테이너를 의미함
- 순차적으로 연산되는 레이어만 있을 경우에는, nn.Sequential 을 통해 순서대로 각 레이어를 작성하면 그대로 실행됨
  - 중간에 activation function 이 적용된다면, activation function 도 순서에 맞게 넣어주면 자동 계산됨

In [25]:
x = torch.ones(4)  # input tensor
y = torch.zeros(3)  # expected output

input_dim = x.size(0)
output_dim = y.size(0)

model = nn.Sequential (
    nn.Linear(input_dim, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, output_dim)    
)

In [26]:
loss_function = nn.MSELoss()
learning_rate = 0.01
nb_epochs = 1000 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):

    y_pred = model(x)
    loss = loss_function(y_pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(loss)
for param in model.parameters():
    print (param)

tensor(1.4440e-10, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[-0.2754, -0.0372, -0.2407,  0.3626],
        [ 0.4701,  0.0925,  0.0651, -0.3780],
        [-0.0094, -0.2228, -0.0772, -0.2660],
        [-0.1651,  0.3736, -0.0696,  0.3334],
        [ 0.0909, -0.2044, -0.2411,  0.4714],
        [-0.2170,  0.1812, -0.4926, -0.4065],
        [-0.2669, -0.4894, -0.2360, -0.2656],
        [-0.3523,  0.4542,  0.2489,  0.0503],
        [-0.3685,  0.3352, -0.4547, -0.1881],
        [-0.2065, -0.2266, -0.4444,  0.4920]], requires_grad=True)
Parameter containing:
tensor([-0.0641, -0.0401,  0.2730, -0.3222,  0.4448, -0.4015,  0.1903,  0.4497,
         0.3051, -0.1508], requires_grad=True)
Parameter containing:
tensor([[-0.2571,  0.3119,  0.3030,  0.2072, -0.1458,  0.1450,  0.0371,  0.2615,
          0.0027,  0.0714],
        [ 0.2061,  0.2504, -0.0309,  0.1212,  0.1219, -0.2075,  0.2148, -0.0964,
         -0.2055, -0.0731],
        [ 0.0305, -0.1485,  0.2762, -0.1629, -0.2989,  0.1144

### SGD 방식 구현

- 랜덤하게 데이터를 섞기 위한 함수
  - torch.randperm(n) : 0 ~ n - 1 까지의 정수를 랜덤하게 섞어서, 순열(배열)을 만들어 줌
  - torch.index_select(텐서객체, 차원번호, 인덱스텐서)
    - 차원번호는 예를 들어, |x| = (3, 4) 에서 0 차원에 해당하는 값은 3 (행으로 이해하면 됨), 1 차원에 해당하는 값은 4 (열로 이해하면 됨)
  - 특정 차원의 나열된 인덱스 번호 순서대로, 데이터를 섞어줌

In [68]:
data1 = torch.randn(3, 4)
print (data1)
indices = torch.tensor([1, 2])
print (indices)
print (torch.index_select(data1, 0, indices))
print (torch.index_select(data1, 1, indices))

tensor([[ 0.2001,  0.0867,  0.6732, -0.7364],
        [-0.5086, -0.7707, -0.5032,  0.4447],
        [ 0.7114, -0.3392, -0.3495, -0.4505]])
tensor([1, 2])
tensor([[-0.5086, -0.7707, -0.5032,  0.4447],
        [ 0.7114, -0.3392, -0.3495, -0.4505]])
tensor([[ 0.0867,  0.6732],
        [-0.7707, -0.5032],
        [-0.3392, -0.3495]])


### 테스트 구현

In [27]:
x = torch.ones(5000, 10) # 10 개의 feature 가 있는 10000 개의 데이터셋
y = torch.zeros(5000, 1)  # 10000 개의 데이터에 대한 실제 신경망에서 예측해야 하는 결과값
learning_rate = 0.01
nb_epochs = 1000 # 1000 epoch 실행 예정
minibatch_size = 256 # Mini-batch 사이즈는 256 으로 정하고, 1 epoch 에 10000 개 데이터를 40개의 Mini-batch 로 나누어 40 iteration 실행

In [28]:
input_dim = x.size(-1) 
output_dim = y.size(-1)

# 보통 hidden layer 는 출력에 가까울 수록 작아지게 설계하는 것이 일반적임 (더 좋은 성능)
model = nn.Sequential (
    nn.Linear(input_dim, 10),
    nn.LeakyReLU(0.1),
    nn.Linear(10, 8),
    nn.LeakyReLU(0.1),
    nn.Linear(8, 6),
    nn.LeakyReLU(0.1),
    nn.Linear(6, output_dim)    
)

loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [47]:
indices = torch.randperm(x.size(0))
print (indices)
x_batch_list = torch.index_select(x, 0, index=indices) # shuffle 된 데이터셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
y_batch_list = torch.index_select(y, 0, index=indices) # shuffle 된 데이터셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언

tensor([ 583,  781, 2536,  ..., 3404, 4193, 3028])


In [48]:
y_batch_list.shape

torch.Size([5000, 1])

In [49]:
x_batch_list = x_batch_list.split(minibatch_size, dim=0)
y_batch_list = y_batch_list.split(minibatch_size, dim=0)
print (len(x_batch_list), len(y_batch_list)) # 5000 / 256 = 19.5 정도 되므로, 256 데이터를 가진 19 개의 Mini-batch 와 나머지 데이터를 가진 1 개의 Mini-batch 로 구성

20 20


- 참고: 파이썬 zip() 내장 함수
  - for 구문에서, 두 개 이상의 리스트 변수를 같은 인덱스 번호의 데이터 별로 묶어서, 튜플로 반환할 때 많이 활용하는 함수

In [52]:
for index in range(nb_epochs):
    indices = torch.randperm(x.size(0))

    x_batch_list = torch.index_select(x, 0, index=indices)
    y_batch_list = torch.index_select(y, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        loss = loss_function(y_minibatch_pred, y_minibatch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
print(loss)
for param in model.parameters():
    print (param)

tensor(2.1684e-15, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.1279,  0.1960,  0.3121, -0.0354, -0.0729, -0.1457,  0.1145, -0.1305,
          0.2323,  0.0436],
        [-0.2136, -0.1330,  0.0991,  0.2277,  0.1681,  0.0313, -0.2268, -0.2854,
          0.2656, -0.0918],
        [ 0.3094, -0.0944,  0.0782, -0.0658, -0.0181,  0.1384, -0.1609, -0.2764,
          0.2483, -0.0946],
        [-0.1138, -0.2362, -0.0691, -0.0637, -0.2234, -0.2830, -0.1459,  0.1672,
         -0.2570, -0.1796],
        [ 0.0538,  0.0277,  0.3042, -0.2546,  0.1551, -0.0498, -0.1253,  0.1937,
         -0.1640,  0.0430],
        [-0.1426,  0.3114, -0.2172,  0.0948,  0.0868,  0.1932,  0.2807,  0.1693,
         -0.2176, -0.1785],
        [ 0.2155, -0.1413,  0.0165, -0.1992,  0.1384, -0.2318, -0.0168, -0.0837,
         -0.0724,  0.1680],
        [-0.2190,  0.2403, -0.2882,  0.0983, -0.1119,  0.1632, -0.0787,  0.3036,
         -0.2573,  0.2525],
        [ 0.0003,  0.0127,  0.2067,  0.2793,  0.2224,  0.22

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>